# Data cleaning 

In [1]:
# Modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load DataFrames
Podcast_Train_df = pd.read_csv('Data/train.csv')
Podcast_Test_df = pd.read_csv('Data/test.csv')

In [3]:
# to make the preprocessing at the same time
Podcast_Train_df['is_train'] = 1
Podcast_Test_df['is_train'] = 0

In [4]:
# join the both DataFrames
Podcast_df = pd.concat([Podcast_Train_df,Podcast_Test_df])

In [5]:
# 1x10^6 rows
Podcast_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000000 entries, 0 to 249999
Data columns (total 13 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   id                           1000000 non-null  int64  
 1   Podcast_Name                 1000000 non-null  object 
 2   Episode_Title                1000000 non-null  object 
 3   Episode_Length_minutes       884171 non-null   float64
 4   Genre                        1000000 non-null  object 
 5   Host_Popularity_percentage   1000000 non-null  float64
 6   Publication_Day              1000000 non-null  object 
 7   Publication_Time             1000000 non-null  object 
 8   Guest_Popularity_percentage  805138 non-null   float64
 9   Number_of_Ads                999999 non-null   float64
 10  Episode_Sentiment            1000000 non-null  object 
 11  Listening_Time_minutes       750000 non-null   float64
 12  is_train                     1000000 non-null  i

## Cleaning process 
* Episode_Title: Change it  with the number
* Episode_Length_minutes: maximum time 121 minutes NaN values average?
* Host_Popularity_percentage: maximum 100%
* Guest_Popularity_percentage: maximum 100%
* Number_of_Ads: only allowed [0, 1, 2, 3], other values removed
* 

In [6]:
Podcast_df.describe()

,id,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes,is_train
count,1000000.000000,8.841710e+05,1000000.000000,805138.000000,999999.000000,750000.000000,1000000.000000
mean,499999.500000,1.532920e+02,59.824048,52.225542,1.350604,45.437406,0.750000
std,288675.278932,8.346900e+04,22.874903,28.449679,2.358272,27.138306,0.433013
min,0.000000,0.000000e+00,1.300000,0.000000,0.000000,0.000000,0.000000
25%,249999.750000,3.574000e+01,39.370000,28.370000,0.000000,23.178350,0.750000
50%,499999.500000,6.387000e+01,60.020000,53.540000,1.000000,43.379460,1.000000
75%,749999.250000,9.408000e+01,79.490000,76.590000,2.000000,64.811580,1.000000
max,999999.000000,7.848626e+07,119.460000,119.910000,2063.000000,119.970000,1.000000


In [7]:
# Ads NaN ->0
Podcast_df["Number_of_Ads"] = Podcast_df["Number_of_Ads"].fillna(0)

In [8]:
# Define the list of correct values
correct_values = [0.00, 1.00, 2.00, 3.00]

# Create a boolean mask to identify erroneous values
# Values that are NOT in the correct list 
erroneous_mask = ~Podcast_df["Number_of_Ads"].isin(correct_values) 
Podcast_df.loc[erroneous_mask, "Number_of_Ads"] = 1.00

In [9]:
# Change the Host_Popularity_percentage maximun by 100
threshold_host = 100.00
Podcast_df.loc[Podcast_df['Host_Popularity_percentage']>threshold_host, 'Host_Popularity_percentage']= threshold_host

In [10]:
# Change the Guest_Popularity_percentage maximun by 100
threshold_Guest = 100.00
Podcast_df.loc[Podcast_df['Guest_Popularity_percentage']>threshold_Guest, 'Guest_Popularity_percentage']= threshold_Guest

In [12]:
# Change the Guest_Popularity_percentage maximun by 100
threshold_min = 121.00
Podcast_df.loc[Podcast_df['Episode_Length_minutes']>threshold_min, 'Episode_Length_minutes']= threshold_min

In [13]:
Podcast_df.describe()

,id,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes,is_train
count,1000000.000000,884171.000000,1000000.000000,805138.000000,1000000.000000,750000.000000,1000000.000000
mean,499999.500000,64.515301,59.823641,52.225207,1.347722,45.437406,0.750000
std,288675.278932,32.964416,22.874055,28.449038,1.111388,27.138306,0.433013
min,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000,0.000000
25%,249999.750000,35.740000,39.370000,28.370000,0.000000,23.178350,0.750000
50%,499999.500000,63.870000,60.020000,53.540000,1.000000,43.379460,1.000000
75%,749999.250000,94.080000,79.490000,76.590000,2.000000,64.811580,1.000000
max,999999.000000,121.000000,100.000000,100.000000,3.000000,119.970000,1.000000


In [14]:
podcast_data_order = pd.DataFrame(columns=['Program','# progam','day','# week day','Time','# in hour','mean','std'])

In [15]:
podcast_data_order

,Program,# progam,day,# week day,Time,# in hour,mean,std


In [16]:
n=0
for podcast in Podcast_df['Podcast_Name'].unique():
    podname_df = Podcast_df[Podcast_df['Podcast_Name'] == podcast]
    feature = podcast
    program_num = len(podname_df)
    
    for day in ['Sunday' ,'Monday' ,'Tuesday' ,'Wednesday','Thursday','Friday', 'Saturday']:
        try:
            pub_day_df = podname_df[podname_df['Publication_Day'] == day ]
            pod_day = day
            pod_day_num = len(pub_day_df)
        except:
            pub_day_df = podname_df
            pod_day = day
            pod_day_num = np.nan

        for hour in ['Night','Morning', 'Afternoon', 'Evening']:
            try:
                pub_hour_df = pub_day_df[pub_day_df['Publication_Time'] == hour ]
                time = hour
                time_num = len(pub_hour_df)
                mean = pub_hour_df['Episode_Length_minutes'].mean()
                std = pub_hour_df['Episode_Length_minutes'].std()
            except:
                pub_hour_df = pub_day_df
                time = hour
                time_num = np.nan
                mean = np.nan
                std = np.nan
                
            podcast_data_order.loc[n] = [feature, program_num, pod_day, pod_day_num, time, time_num, mean, std]
            n = n +1
        n= n +4
    n=n +7

In [18]:
podcast_data_order.describe()

,# progam,# week day,# in hour,mean,std
count,1344.000000,1344.000000,1344.000000,1344.000000,1344.000000
mean,20833.333333,2976.190476,744.047619,64.498286,32.907894
std,3718.647693,573.680482,154.181327,2.551742,1.010480
min,12307.000000,1681.000000,353.000000,52.152857,29.269114
25%,17822.500000,2521.500000,629.000000,63.016910,32.240054
50%,20649.000000,2922.000000,726.000000,64.562751,32.844947
75%,23243.500000,3402.750000,845.250000,66.140804,33.555257
max,30400.000000,4666.000000,1257.000000,72.077232,37.229366


In [19]:
Podcast_df["Episode_Length_minutes"] = Podcast_df["Episode_Length_minutes"].fillna(Podcast_df["Episode_Length_minutes"].mean())

In [20]:
# Change episode number to a number
Podcast_df["Episode_Number"] = Podcast_df["Episode_Title"].str.extract(r"(\d+)").astype(float)
Podcast_df['Guest_Popularity_percentage'] = Podcast_df['Guest_Popularity_percentage'].fillna(0) 

In [21]:
Podcast_df.describe()

,id,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes,is_train,Episode_Number
count,1000000.000000,1000000.000000,1000000.000000,1000000.000000,1000000.000000,750000.000000,1000000.000000,1000000.000000
mean,499999.500000,64.515301,59.823641,42.048499,1.347722,45.437406,0.750000,51.439605
std,288675.278932,30.996559,22.874055,32.856540,1.111388,27.138306,0.433013,28.077424
min,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,249999.750000,39.400000,39.370000,7.840000,0.000000,23.178350,0.750000,28.000000
50%,499999.500000,64.515301,60.020000,42.170000,1.000000,43.379460,1.000000,52.000000
75%,749999.250000,90.340000,79.490000,71.030000,2.000000,64.811580,1.000000,75.000000
max,999999.000000,121.000000,100.000000,100.000000,3.000000,119.970000,1.000000,100.000000


In [22]:
# Drop unused
Podcast_df.drop(columns=["Episode_Title"], inplace=True)

In [24]:
# Split back Podcast_df
Podcast_Train_df = Podcast_df[Podcast_df['is_train']== 1].drop(columns=["is_train"])
Podcast_Test_df = Podcast_df[Podcast_df['is_train']== 0].drop(columns=["is_train", "Listening_Time_minutes"])

In [26]:
Podcast_Test_df.describe()

,id,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Episode_Number
count,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000,250000.000000
mean,874999.500000,64.544125,59.716117,41.997790,1.347252,51.420988
std,72168.927986,31.001667,22.879264,32.851138,1.112666,28.052862
min,750000.000000,2.470000,2.490000,0.000000,0.000000,1.000000
25%,812499.750000,39.370000,39.250000,7.790000,0.000000,28.000000
50%,874999.500000,64.515301,59.900000,42.090000,1.000000,52.000000
75%,937499.250000,90.510000,79.390000,70.990000,2.000000,75.000000
max,999999.000000,121.000000,100.000000,100.000000,3.000000,100.000000


In [27]:
Podcast_Train_df.to_csv('Data/Podcast_train_clean.csv', index=False)

In [28]:
Podcast_Test_df.to_csv('Data/Podcast_test_clean.csv', index=False)